In [1]:
import scrapy
import scrapy.crawler as crawler
from scrapy.utils.log import configure_logging
from multiprocessing import Process, Queue
from twisted.internet import reactor
from pydispatch import dispatcher
import logging
import json

logging.basicConfig(level=logging.ERROR)
logger = logging.getLogger('nlp')
logger.setLevel(logging.INFO)

# Fetch sentences

In [2]:
def get_web_data(function):
    def f(q):
        try:
            json_data = function()
            q.put(json.dumps(json_data))
        except Exception as e:
            q.put(e)
    q = Queue()
    p = Process(target=f, args=(q,))
    p.start()
    result = q.get()
    p.join()
    try:
        json_data = json.loads(result)
    except:
        raise result
    return json_data

## USA goverment RSS channels

In [3]:
def get_usa_links():
    links_storage = {'links' : []}
    
    class LinkCheckerSpider2(scrapy.Spider):
        name = "link_checker2"
        start_urls = ['https://www.state.gov/rss-feeds/']
        
        def parse(self, response):
            a_selectors = response.xpath("//a")
            for selector in a_selectors:
                link = selector.xpath("@href").extract_first()
                if 'rss' in link:
                    links_storage['links'].append(link)

    runner = crawler.CrawlerRunner()
    deferred = runner.crawl(LinkCheckerSpider2)
    deferred.addBoth(lambda _: reactor.stop())
    reactor.run()
    return links_storage

# def get_usa_rss_data():
    

IndentationError: expected an indented block (4121935701.py, line 19)

In [ ]:

#     print(links_storage)

# def get_data_from_links(links_storage):
    

usa_links_storage = get_web_data(get_usa_links)

In [ ]:
usa_links_storage